In [ ]:
from collections import  defaultdict

with open("Data/ES/train") as f:
    es = f.read().splitlines()

with open("Data/RU/train") as f:
    ru = f.read().splitlines()

In [ ]:
def estimate_emission_params(train_data,k=1):

    word_sentiment_counts = defaultdict(lambda: defaultdict(int))
    sentiment_counts =  defaultdict(int)
    emission_params = {}
    
    #getting the count(y) and count(y --> x)
    for sentence in train_data:
        try:
            if(sentence!=""):
                x, label = sentence.split(" ")
        except:
            continue

        sentiment_counts[label]+=1

        word_sentiment_counts[label][x]+=1



    #calculating the emission parameters

    for key in word_sentiment_counts:
        for word in word_sentiment_counts[key]:
            emission_params[(word,key)] = word_sentiment_counts[key][word]/(sentiment_counts[key])
    return emission_params, sentiment_counts

In [ ]:
#Getting the data from ES 
# train_es = read_data('./Data/ES/train')

def estimate_emission_params_unk(x,y,k=1):
    if x=="#UNK":
        return k/(y+k)


    # count_y_to_x, count_y = 0, 0
    # for i in range(len(tags)):
    #     if tags[i] == y:
    #         count_y += 1
    #         if words[i] == x:
    #             count_y_to_x += 1
    # return (count_y_to_x) / (count_y + k)
    
    # word_sentiment_counts = defaultdict(lambda: defaultdict(int))
    # sentiment_counts = defaultdict(lambda: 1)
    # emission_params = {}
    
    # #getting the count(y) and count(y --> x)
    # for sentence in train_data:
    #     try:
    #         if(sentence!=""):
    #             x, label = sentence.split(" ")
    #     except:
    #         continue

    #     sentiment_counts[label]+=1

    #     word_sentiment_counts[label][x]+=1


    # #changing count(y->x) when x="#UNK"
    # for key in word_sentiment_counts:
    #     word_sentiment_counts[key]["#UNK#"]=k


    # #calculating the emission parameters

    # for key in word_sentiment_counts:
    #     for word in word_sentiment_counts[key]:
    #         emission_params[(word,key)] = word_sentiment_counts[key][word]/(sentiment_counts[key]+k)
    # return emission_params


In [ ]:
# results of training using es and ru datasets for part 1
result_es = estimate_emission_params(es)
result_ru = estimate_emission_params(ru)
# # results of training using es and ru datasets for part 2
# result_es_unk = estimate_emission_params_unk(es)
# result_ru_unk = estimate_emission_params_unk(ru)

PART 3

In [ ]:
tags = ["B-neutral", "B-positive", "B-negative", "I-neutral", "I-positive", "I-negative", "O"]

result_es, count_y = estimate_emission_params(es)
y_optimal = defaultdict(str)
for word, sentiment in result_es:
    max1 = []
    for tag in tags:
        if (word, tag) in result_es:
            max1.append((tag, result_es[(word, tag)]))
    y_optimal[word] = max(max1, key=lambda x: x[1])[0]

In [ ]:
with open('./Data/ES/dev.in') as f:
    x = f.readlines()
with open('./EvalScript/dev.p1.out', 'w+') as f:
        
    for i in x:
        if i == '\n':
            f.write(i)
        else:
            word = i.strip()
            if word in y_optimal:
                f.write(word + ' ' + y_optimal[word] + '\n')
            else:
                # if the word is an unk, use the emission_unk function to find the y_max and write it into the file
                y_max_prob=0
                y_max=""
                for y in tags:
                    ep=estimate_emission_params_unk("#UNK",y,count_y)
                    if ep>y_max_prob:
                        y_max_prob=ep
                        y_max=y
                f.write(word +" "+y_max+"\n")
        

In [ ]:
# Calculating the Precision, Recall and F score for the given data
def scores(labels, correct_labels):
    correct = 0
    total_predicted = 0
    for word in labels:
        if labels[word] == correct_labels[word]:
            correct += 1
        total_predicted += 1
        
    precision = correct / total


def F_score(recall,precision):
    v1=1/precision
    v2=1/recall
    f=2/(v1+v2)